In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException, JavascriptException, InvalidArgumentException, TimeoutException 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import tqdm
import multiprocessing

In [ ]:
cd_path = #путь до chromedriver

In [ ]:
class SchoolParser:
    def __init__(self, query_list : list = [], cd_path=cd_path):
        '''cd_path - путь до chromedriver
        query_list - список запросов
        res - переменная, в которую сохраняются адреса
        '''
        self.cd_path = cd_path
        self.res = []
        self.query_list = query_list
        
        
    def set_query_list(self, query_list : list ):
        '''Просто сеттер для query_list, который используется в parse_schools'''
        self.query_list = query_list
        print('Query List Set')
        
        
    def parse_schools(self):
        '''Скрипт для парсинга запросов, использует яндекс карты, итерируется по self.query_list'''
        if not self.query_list:
            raise 'No Query List Found'
        browser = webdriver.Chrome(executable_path=self.cd_path)
        browser.execute_script("window.open('about:blank', '_blank')")
        for n in tqdm.tqdm(range(len(self.query_list)), leave=False):
            query = self.query_list[n]
            browser.switch_to.window(browser.window_handles[-1])
            browser.get('https://yandex.ru/maps')
            search = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input._bold')))
            search.send_keys(query)
            search.send_keys(Keys.RETURN)
            try:
                WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[9]/div[2]/div[1]/div[1]/div[1]/div/div[1]/div/div/ul/li[1]/div'))).click()
                element = WebDriverWait(browser, 5).until(lambda x: x.find_element(By.XPATH, '//div[@role="link" and (@class="business-contacts-view__address-link" or @class="business-contacts-view__link _new-address") and @tabindex="0"]'))
                aria_label = element.get_attribute("aria-label")
                self.res.append(aria_label)

            except (TimeoutException, NoSuchElementException):
                try:
                    #browser.implicitly_wait(4)
                    #element = browser.find_element_by_css_selector('.business-contacts-view__address-link, .business-contacts-view__link._new-address')
                    element = WebDriverWait(browser, 4).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.business-contacts-view__address-link, .business-contacts-view__link._new-address')))
                    aria_label = element.get_attribute("aria-label")
                    self.res.append(aria_label)
                #except NoSuchElementException:
                except TimeoutException:
                    self.res.append(None)
                    continue
        

    def set_queries(self, df):
        '''Вынимает из датафрейма нужные для формирования запроса колонки и преобразует их
        edu_flag - флаг того, что человек пришел из школы
        query - переменная, из которой берутся запросы для яндекс карт
        '''
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        df[['country_reg', 'school', 'city_reg']] = df[['country_reg', 'school', 'city_reg']].apply(lambda x: x.str.lower())
        df['edu_flag'] = df.apply(lambda x: x['уровень_подготовки'] in ['Бакалавриат', 'Специалитет', 'Академический бакалавриат'] or x['country_reg'] != 'россия', axis=1)
        df.fillna('', inplace=True)
        df['query'] = df.apply(lambda x: x['country_reg'] + ' ' + x['city_reg'] + ' ' + x['school'] if x["edu_flag"] else x['country_reg'] + ' ' + x['school'], axis=1)
        self.query_list = df['query'].tolist()
        print('Query List Set')

        
    def merge_df(self, df, save_path : str, save: bool = False):
        '''Соединяет результат res в переменную df['school_address']
        save_path - путь в который сохранится ответ, записывается в виде r'...\...\parser_res(или любое другое имя файла).csv'
        df['school_city'] будет брать последний элемент из адреса(не всегда работает для иностранцев, не особо важное уточнение)
        Принтит в том числе количество людей из МСК и СПб
        '''
        df['school_address'] = self.parser.res
        df['school_city'] = df['school_address'].apply(lambda x: x.split(',')[-1].strip() if x else None)
        print(f'Количество людей из СПб и МСК: {len(df[(df["school_city"] == "Москва") | (df["school_city"] == "Санкт-Петербург")])}')
        if save:
              df.to_csv(save_path, index=False)
        

In [ ]:
#Создаем объект класса
parser = SchoolParser()
#Даем ему датафрейм
parser.set_queries(df)
#Запускаем скрипт
parser.parse_schools()
#Ждем, пока tqdm bar не наполнится